# Time Series Cross Validation: Holt-Winters Exponential Smoothing with additive errors and seasonality.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')


#forecast error metrics
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

import statsmodels as sm
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing

import warnings
warnings.filterwarnings('ignore')

In [41]:
print(sm.__version__)

0.11.0


In [42]:
#ensemble learning
from amb_forecast.ensemble import (Ensemble, UnweightedVote)

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [5]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'hw'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)

In [11]:
train.shape

(1279, 9)

In [12]:
val.shape

(549, 9)

#  Test fitting and predicting with model.

The class below is a 'wrapper' class that provides the same interfacew for all methods and works in the time series cross valiation code. 

In [13]:
class ExponentialSmoothingWrapper:
    '''
    Facade for statsmodels exponential smoothing models.  This wrapper
    provides a common interface for all models and allow interop with
    the custom time series cross validation code.
    '''
    def __init__(self, trend=False, damped_trend=False, seasonal=None):
        self._trend = trend
        self._seasonal= seasonal
        self._damped_trend = damped_trend

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues

    def fit(self, train):
        '''
        Fit the model
        
        Parameters:
        train: array-like
            time series to fit.
        '''
        self._model = ExponentialSmoothing(endog=train,
                                          trend=self._trend, 
                                          damped_trend=self._damped_trend,
                                          seasonal=self._seasonal)
        self._fitted = self._model.fit()
        self._t = len(train)
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        Forecast the time series from the final point in the fitted series.
        
        Parameters:
        ----------
        
        horizon: int
            steps ahead to forecast 
            
        return_conf_int: bool, optional (default=False)
            Return prediction interval?  
            
        alpha: float
            Used if return_conf_int=True. 100(1-alpha) interval.
        '''
        
        forecast = self._fitted.get_forecast(horizon)
        
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)

# Example fitting and prediction with comb

In [17]:
model_1 = ExponentialSmoothingWrapper(trend=True, damped_trend=True, 
                                      seasonal=7)

In [18]:
estimators = {'shw': model_1}
ens = Ensemble(estimators, UnweightedVote())

In [19]:
ens.fit(train[REGION])

In [20]:
H = 5
ens_preds = ens.predict(horizon=H)

In [21]:
ens_preds, pi = ens.predict(horizon=H, return_conf_int=True)

In [22]:
ens_preds

array([2244.63585174, 2251.76403655, 2142.71857983, 2085.26940713,
       2072.41902494])

In [23]:
pi

array([[2142.32799116, 2346.94371233],
       [2144.42299751, 2359.10507558],
       [2030.56812545, 2254.8690342 ],
       [1968.50601141, 2202.03280285],
       [1951.21700718, 2193.6210427 ]])

## Time Series Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [24]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    error_func - function to measure forecast error

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days

    step -- step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array - vector of forecast errors from the CVs.
    '''
    cv_preds = [] #mean forecast
    cv_actuals = [] # actuals 
    cv_pis = [] #prediction intervals
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)
        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [25]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [26]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

### Get model and conduct tscv.

In [27]:
def get_model():
    '''
    Create ensemble model
    '''
    model_1 = ExponentialSmoothingWrapper(trend=True, damped_trend=True, 
                                         seasonal=7)
    estimators = {'hw': model_1}
    return Ensemble(estimators, UnweightedVote())
    

In [28]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_model()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [29]:
cv_preds, cv_test, cv_intervals = results
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2.870949,3.232631,3.387609,3.498032,3.658897,3.823331,3.935145,4.031783,4.143843,4.211268,4.271285,4.351160,5.124295
std,1.244457,1.436743,1.398462,1.295544,1.314220,1.315240,1.312934,1.280611,1.304054,1.199576,1.108240,1.129762,1.672828
min,1.098028,1.629225,2.038119,2.071345,2.017373,2.420873,2.358999,2.346259,2.408038,2.480246,2.632331,2.889536,3.299821
25%,2.034193,2.231086,2.691763,2.848122,2.841323,3.081003,3.077843,3.285958,3.378797,3.509652,3.695005,3.675022,3.736102
50%,2.448456,2.721411,2.858200,3.167727,3.362226,3.652577,3.709987,3.693752,3.884546,3.995116,4.079412,4.290042,4.927938
75%,3.534764,3.797758,3.582721,3.594094,4.195626,4.276492,4.697929,4.468538,4.426819,4.679833,4.828968,4.740457,6.214774
max,6.752309,8.201948,8.701131,8.447591,8.850626,8.891030,8.859678,8.538516,8.854046,8.505218,8.226571,8.566002,10.585837


In [30]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-hw_smape.csv


In [31]:
#CV point predictions rmse
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,77.217641,88.148500,94.071881,98.203561,103.501679,108.349871,111.909771,114.945485,118.453340,121.074300,123.374606,126.188281,144.198494
std,36.806688,42.642933,41.965376,39.832151,39.641860,39.092541,38.718744,38.350867,38.399028,35.645921,32.645715,32.063856,36.839152
min,35.134065,48.789351,54.457237,58.721873,57.558830,68.618690,66.680558,66.064628,67.897861,69.155446,73.125610,80.318381,103.943216
25%,55.302277,62.479244,73.175423,76.085373,76.783474,82.575906,83.075776,88.985958,91.386495,96.161811,100.079498,102.447186,114.582753
50%,65.055825,78.340805,80.751215,83.919310,89.293133,98.463373,99.961035,98.069393,103.865521,111.686042,122.284916,124.630255,140.189152
75%,87.165244,98.544999,97.261570,98.308341,110.602700,122.174997,142.693911,144.353974,140.136311,140.358974,139.693738,144.184777,166.916891
max,213.739761,225.596805,232.153538,222.576604,229.130603,227.959312,226.489084,219.373298,225.908773,219.624089,213.551635,222.858060,265.614654


In [32]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-hw_rmse.csv


In [33]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.776390,0.876827,0.921432,0.953308,0.998179,1.044021,1.075422,1.102048,1.132846,1.151381,1.168091,1.190157,1.387984
std,0.374427,0.432041,0.421459,0.394370,0.398583,0.398332,0.395551,0.384907,0.388749,0.357200,0.329212,0.331624,0.462209
min,0.315950,0.469174,0.545216,0.557763,0.545605,0.651639,0.635532,0.632770,0.650322,0.670474,0.712974,0.785930,0.899954
25%,0.537860,0.596261,0.696459,0.734216,0.759401,0.798927,0.791375,0.856064,0.874999,0.943435,0.979841,0.997156,1.014067
50%,0.651737,0.730453,0.777270,0.834931,0.879109,0.945330,0.991317,1.012393,1.045180,1.085842,1.106382,1.129539,1.327436
75%,0.922192,1.030079,0.944162,0.962470,1.135749,1.231902,1.331564,1.285021,1.230079,1.279489,1.322365,1.322210,1.661264
max,2.070509,2.407964,2.525015,2.447888,2.552795,2.560451,2.547150,2.453349,2.537378,2.437239,2.359933,2.452495,2.985912


In [34]:
#output rmse
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-hw_mase.csv


In [35]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.851852,0.857143,0.871252,0.884921,0.889947,0.895944,0.903250,0.909392,0.912992,0.917989,0.923040,0.925485,0.973212
std,0.191996,0.183717,0.164329,0.144677,0.144323,0.136958,0.119883,0.104703,0.104028,0.094312,0.085763,0.081253,0.022132
min,0.285714,0.214286,0.238095,0.321429,0.285714,0.309524,0.387755,0.464286,0.460317,0.514286,0.558442,0.571429,0.876712
25%,0.857143,0.785714,0.833333,0.857143,0.857143,0.857143,0.857143,0.875000,0.880952,0.885714,0.896104,0.904762,0.975342
50%,0.857143,0.928571,0.952381,0.928571,0.914286,0.928571,0.938776,0.946429,0.936508,0.928571,0.935065,0.928571,0.978082
75%,1.000000,1.000000,0.952381,0.964286,0.971429,0.976190,0.979592,0.982143,0.984127,0.985714,0.980519,0.976190,0.980822
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.991781


In [36]:
#output 80% PI coverage
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-hw_coverage_80.csv


### Rerun for 95% PI coverage

In [37]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_model()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [38]:
#95% PIs
cv_preds, cv_test, cv_intervals = results
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.973545,0.962963,0.964727,0.970899,0.973545,0.976190,0.978080,0.979497,0.980600,0.982011,0.983165,0.984127,0.994521
std,0.069049,0.105520,0.101095,0.076108,0.065789,0.054853,0.046992,0.041052,0.036921,0.033053,0.029880,0.027227,0.005635
min,0.714286,0.571429,0.523810,0.642857,0.685714,0.738095,0.775510,0.803571,0.825397,0.842857,0.857143,0.869048,0.969863
25%,1.000000,1.000000,1.000000,0.982143,0.971429,0.976190,0.969388,0.964286,0.968254,0.971429,0.974026,0.976190,0.994521
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.988095,0.997260
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.997260
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.997260


In [39]:
#output 95% PI coverage
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-hw_coverage_95.csv


# End